In [16]:
from scipy.optimize import minimize
import numpy as np

In [17]:
# error function parameters  modes
eta = 1.62843649e-02
'''rho = pm.Uniform("rho", lower=0, upper=10)
beta_L = pm.Uniform("beta_L", lower=0, upper=3)
beta_H = pm.Uniform("beta_H", lower=0, upper=6)'''
rho = 8.65281578e+02
beta_L = 1.46743745e+00
beta_H = 1.13711515e+00
# rho_2 = pm.Uniform("rho_2", lower=0, upper=1)
beta_H2 = 1.97072336e-01
# alpha_1 = pm.Normal("alpha_1", mu=10, sigma=5)
alpha_L = 2.06906956e+01 
alpha_1 = 1.16169405e+01
alpha_2 = -2.66444760e+00

sigma_lg_Phi = 7.68996799e-02   # standard deviation of the log of the error function

# computational cost
C_L = (0.02092 + 0.04517)*56 # CPU hours # 1.97  # C_L1 + C_L2 in CPU hours
C_H = 4.348*56  # 230.59

q = C_H / C_L

# objective error
err_0 = .01 # goal error


In [31]:
q

65.78907550310181

In [18]:
750*(.994 + 1.845)*1200/56

45626.78571428572

In [19]:
def err_func(n_LF, n_HF):
    Phi = eta * (rho / ( (n_LF + n_HF) * (n_LF + alpha_L)**(beta_L-1) * (n_HF + alpha_1)**(beta_H-1) ) + 1 / (alpha_2 + n_HF)**(beta_H2))
    # median_Phi = 10**lg_Phi
    return Phi

def err_mean(n_LF, n_HF):
    gamma = np.exp(1/2 * (sigma_lg_Phi * np.log(10))**2)
    return gamma * err_func(n_LF, n_HF)

def comp_bud(n_L, n_H):
    return C_L * n_L + C_H * n_H

In [20]:


# # Define the objective function
def objective(vars):
    n_L, n_H = vars
    return comp_bud(n_L, n_H)

# Define the constraint function
def constraint(vars):
    n_L, n_H = vars
    return err_mean(n_L,n_H) - err_0

# Initial guess for the variables
initial_guess = [400, 10]

# Define the constraint dictionary
con = {'type': 'eq', 'fun': constraint}

# Perform the optimization
result = minimize(objective, initial_guess, constraints=[con])

# Extract the optimal values of x and y
x_opt, y_opt = result.x

print(f"Optimal values: x = {x_opt}, y = {y_opt}")
print(f"Function value at optimal points: f(x, y) = {objective((x_opt, y_opt))}")

Optimal values: x = 614.8887480172884, y = 21.07246185648742
Function value at optimal points: f(x, y) = 7406.619444474314


In [22]:
comp_bud(x_opt, y_opt) / 56 * 1200 

158713.2738101639

In [23]:
comp_bud(750, 10) / 56 * 1200 

111657.00000000001

In [24]:
n_LF = 564
n_HF = 21

In [25]:
err_mean(n_LF,n_HF)

0.010096500805804777

In [26]:
comp_bud(n_LF, n_HF) / 56 * 1200

154299.312

In [27]:
err_mean(0, 66.3)

0.03612422365359971

In [28]:
comp_bud(0, 66.3) / 56 * 1200

345926.88

In [29]:
err_func(10000, 0)

(0.010946269869192593-0.007790451302149712j)

In [30]:
# error function at integer n_LF and n_HF
n_LF = 78
n_HF = 9
phi = eta * (rho * n_LF**(-beta_L) + n_HF**(-beta_H))
print("phi at n_LF = %d, n_HF = %d:" % (n_LF, n_HF), "%.2f" % (phi * 100) + "%")

# computational cost at integer n_LF and n_HF
cpus = 56 # frontera
C = C_L * n_LF + C_H * n_HF
print("C at n_LF = %d, n_HF = %d:" % (n_LF, n_HF), C, "CPU hours", "= %.2f" % (C/cpus), "node hours")

phi at n_LF = 78, n_HF = 9: 2.49%
C at n_LF = 78, n_HF = 9: 2480.07312 CPU hours = 44.29 node hours
